In [10]:
import os
import keras.optimizers
import numpy as np
%matplotlib inline
from transformers import AutoImageProcessor, ViTImageProcessor
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

import cv2
import datasets
from transformers import DefaultDataCollator
from transformers import TFViTForImageClassification, create_optimizer, TFCvtForImageClassification
from transformers import CvtConfig, CvtModel
import matplotlib.pyplot as plt


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [11]:
def create_image_folder_dataset(root_path):
    """creates `Dataset` from image folder structure"""

    # get class names by folders names
    _CLASS_NAMES = os.listdir(root_path)
    # defines `datasets` features`
    features = datasets.Features({
        "img": datasets.Image(),
        "label": datasets.features.ClassLabel(names=_CLASS_NAMES),
    })
    # temp list holding datapoints for creation
    img_data_files = []
    label_data_files = []
    # load images into list for creation
    for img_class in os.listdir(root_path):
        for img in os.listdir(os.path.join(root_path, img_class)):
            path_ = os.path.join(root_path, img_class, img)
            img_data_files.append(path_)
            label_data_files.append(img_class)
    # create dataset
    ds = datasets.Dataset.from_dict({"img": img_data_files, "label": label_data_files}, features=features)
    return ds


train_imgs = create_image_folder_dataset("train")
img_class_labels = train_imgs.features["label"].names

In [12]:
model_id = "google/vit-base-patch16-224-in21k"
#model_id = "microsoft/cvt-13"
feature_extractor = ViTImageProcessor.from_pretrained(model_id)
feature_extractor.size = {"width":224,"height":224}
# learn more about data augmentation here: https://www.tensorflow.org/tutorials/images/data_augmentation
data_augmentation = keras.Sequential(
    [
        layers.Resizing(1024, 1024),
        layers.CenterCrop(900, 900),
        layers.experimental.preprocessing.RandomCrop(600, 600),
        layers.RandomBrightness(factor=0.2),
        layers.RandomContrast(factor=0.2),

        layers.Rescaling(1/255),
        layers.Resizing(224, 224),
        #layers.RandomZoom(height_factor=(0,0.15), width_factor=(0,0.15), fill_mode="constant", ),
        layers.RandomFlip("horizontal"),

        #layers.RandomRotation(factor=0.2, fill_mode="constant", fill_value=0),

    ],
    name="data_augmentation",
)
data_resizing = keras.Sequential(
    [
        layers.Resizing(1024, 1024),
        layers.CenterCrop(900, 900),
        layers.experimental.preprocessing.RandomCrop(450, 450),
        layers.Rescaling(1/255),
        layers.Resizing(224, 224),
    ],
    name="data_resizing",
)
def load_ben_color(image):
    sigmaX=10
    image=cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), sigmaX) ,-4 ,128)
    return image
# use keras image data augementation processing
def augmentation(examples):

    inputs = {"pixel_values":[data_augmentation(np.array(load_ben_color(np.array(img)))) for img in examples['img']], "labels":examples["label"]}
    #inputs["pixel_values"] = np.array(inputs["pixel_values"]).swapaxes(1,3)
    #raise Exception(str(tf.reduce_min(inputs["pixel_values"])) + " " + str(tf.reduce_max(inputs["pixel_values"])))

    return inputs


# basic processing (only resizing)
def process(examples):
    inputs = {"pixel_values":[data_resizing(np.array(load_ben_color(np.array(img)))) for img in examples['img']], "labels":examples["label"]}
    #inputs["pixel_values"] = np.array(inputs["pixel_values"]).swapaxes(1,3)
    return inputs

In [13]:
test_size = .1
train_val_set = train_imgs.train_test_split(test_size=test_size)
train_val_set["test"] = train_val_set["test"].with_transform(process)
train_val_set["train"] = train_val_set["train"].with_transform(augmentation)
#train_val_set["train"] = train_val_set["train"].with_transform(augmentation)

from transformers import TFViTForImageClassification

labels = train_val_set['train'].features['label'].names

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50, MobileNetV2, ResNet101V2, ResNet152V2
from tensorflow.keras.applications.resnet50 import preprocess_input

# ResNet50 model
#resnet_50 = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
resnet_50 = ResNet152V2(weights='imagenet',
                   include_top=False,
                   input_shape=(224, 224, 3))
#resnet_50.trainable = False
#build the entire model
x = resnet_50.output
x = layers.GlobalMaxPooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
predictions = layers.Dense(5, activation='softmax')(x)
model = Model(inputs=resnet_50.input, outputs=predictions)
for layer in resnet_50.layers:#[:-20]:
   layer.trainable = False
"""model = TFViTForImageClassification.from_pretrained(
    model_id,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)"""

#model = TFCvtForImageClassification.from_pretrained(
#    model_id,
#    )
#model.classifier = tf.keras.layers.Dense(5)
#model.num_labels = 5

'model = TFViTForImageClassification.from_pretrained(\n    model_id,\n    num_labels=len(labels),\n    id2label={str(i): c for i, c in enumerate(labels)},\n    label2id={c: str(i) for i, c in enumerate(labels)}\n)'

In [14]:
num_train_epochs = 5
train_batch_size = 64
eval_batch_size = 64
learning_rate = 0.001
weight_decay_rate = 0.01
num_warmup_steps = 0
output_dir = model_id.split("/")[1]
hub_model_id = f'{model_id.split("/")[1]}-eyes'
fp16 = True
data_collator = DefaultDataCollator(return_tensors="tf")

# converting our train dataset to tf.data.Dataset
tf_train_dataset = train_val_set["train"].to_tf_dataset(
    columns=['pixel_values'],
    label_cols=["labels"],
    shuffle=True,
    batch_size=train_batch_size,
    collate_fn=data_collator)

# converting our test dataset to tf.data.Dataset
tf_eval_dataset = train_val_set["test"].to_tf_dataset(
    columns=['pixel_values'],
    label_cols=["labels"],
    shuffle=True,
    batch_size=eval_batch_size,
    collate_fn=data_collator)

C:\Users\mettn\miniconda3\envs\tf\lib\site-packages\datasets\arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


InvalidArgumentError: Exception encountered when calling layer "center_crop_2" "                 f"(type CenterCrop).

{{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} expected 4 inputs, got 276 [Op:StridedSlice] name: data_augmentation/center_crop_2/strided_slice/

Call arguments received by layer "center_crop_2" "                 f"(type CenterCrop):
  • inputs=tf.Tensor(shape=(1024, 1024, 3), dtype=float32)

In [ ]:
"""for x, y in tf_train_dataset:
    print(np.array(x[0]).min(), np.array(x[0]).max())
    test = np.array(x[0])
    print(test.shape)
    plt.imshow(x[0])
    plt.show()"""

In [ ]:
class model_per_epoch(keras.callbacks.Callback):
    def __init__(self, model,filepath,save_best_only):
        self.filepath=filepath
        self.model=model
        self.save_best_only=save_best_only
        self.lowest_loss=np.inf
        self.best_weights=self.model.get_weights()
    def on_epoch_end(self,epoch, logs=None):
        v_loss=logs.get('val_loss')
        if v_loss< self.lowest_loss:
            self.lowest_loss =v_loss
            self.best_weights=self.model.get_weights()
            self.best_epoch=epoch +1
            self.model.set_weights(self.best_weights)
            name= str(self.best_epoch) +'-' + str(self.lowest_loss)[:str(self.lowest_loss).rfind('.')+3] + '.h5'
            file_id=os.path.join(self.filepath, name)
            self.model.save_weights(file_id)
        if self.save_best_only==False:
            name= str(epoch) +'-' + str(v_loss)[:str(v_loss).rfind('.')+3] + '.h5'
            file_id=os.path.join(self.filepath, name)
            self.model.save(file_id)
    def on_train_end(self, logs=None):
        if self.save_best_only == True:
            self.model.set_weights(self.best_weights)
            name= str(self.best_epoch) +'-' + str(self.lowest_loss)[:str(self.lowest_loss).rfind('.')+3] + '.h5'
            file_id=os.path.join(self.filepath, name)
            self.model.save_weights(file_id)
            print(' model is returned with best weights from epoch ', self.best_epoch)

save_dir=r''

In [ ]:

metrics=[
    tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
]
callbacks=[tf.keras.callbacks.EarlyStopping(patience=1),
           model_per_epoch(model, save_dir, True)]
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics="accuracy",)

In [ ]:
history = model.fit(
    tf_train_dataset.prefetch(20),
    validation_data=tf_eval_dataset.prefetch(20),
    callbacks=callbacks,
    epochs=100,
)

In [ ]:
model.save_weights("transformer_weights.h5")